### Problem Statement

We gave two simple heuristics for the 8-puzzle: Manhattan distance and misplaced tiles. Several heuristics in the literature purport to improve on this-see, for example, Nilsson:1971, Mostow+Prieditis:1989, and Hansson+al:1992. Test these claims by implementing the heuristics and comparing the performance of the resulting algorithms.

# The Puzzle

In [ ]:
from __future__ import annotations
from datetime import datetime

In [ ]:
# A class to define the node object
class Node:
    def __init__(self,parent,data,level,fval):
        self.data = data
        self.level = level
        self.fval = fval
        self.parent = parent

    # Generate child nodes by moving the blank space
    # in one of the four directions
    def generate_child(self,parent:Node):
        x,y = self.find(self.data,'_')
        # val_list contains position values for the blank space to move to 
        val_list = [[x,y-1],[x,y+1],[x-1,y],[x+1,y]]
        children = []
        for i in val_list:
            child = self.shuffle(self.data,x,y,i[0],i[1])
            if child is not None:
                child_node = Node(parent,child,self.level+1,0)
                children.append(child_node)
        return children
    
    # To move the position of the blank
    def shuffle(self,puz,x1,y1,x2,y2):
        if x2 >= 0 and x2 < len(self.data) and y2 >= 0 and y2 < len(self.data):
            temp_puz = []
            temp_puz = self.copy(puz)
            temp = temp_puz[x2][y2]
            temp_puz[x2][y2] = temp_puz[x1][y1]
            temp_puz[x1][y1] = temp
            return temp_puz
        else:
            return None

    def copy(self,root):
        temp = []
        for i in root:
            t = []
            for j in i:
                t.append(j)
            temp.append(t)
        return temp    
    
    # To find the blank space
    def find(self,puz,x):
        for i in range(0,len(self.data)):
            for j in range(0,len(self.data)):
                if puz[i][j] == x:
                    return i,j

In [ ]:
class Puzzle:
    # Initialize with size (3 as in 3X3 for 8 puzzle), open and closed list (for node handling),
    # and heuristic to be used
    def __init__(self,size,heuristic):
        self.n = size
        self.open = []
        self.closed = []
        self.heuristic = heuristic

    # To accept the start state
    # Click enter after each row, hit space between elements
    def accept(self):
        puz = []
        for i in range(0,self.n):
            temp = input().split(" ")
            puz.append(temp)
        return puz

    # Function to return the fvalue for each state
    def f(self,start,goal):
        if self.heuristic == "Misplaced Tiles":
            return self.MisplacedTiles(start.data,goal)+start.level
        elif self.heuristic == "Manhattan Distance":
            return self.ManhattanDist(start.data,goal)+start.level
        elif self.heuristic == "Nilsson Sequence":
            return self.Nilsson(start.data,goal)+start.level
        elif self.heuristic == "Out Of Row and Column":
            return self.OutOfRowColumn(start.data,goal)+start.level
        elif self.heuristic == "Linear Conflicts":
            return self.LinearConflicts(start.data,goal)+start.level

    # The Misplaced Tiles heuristic
    def MisplacedTiles(self,mat,goal):
        count = 0
        for i in range(0,self.n):
            for j in range(0,self.n):
                if mat[i][j] != goal[i][j] and mat[i][j] != '_':
                    count += 1
        return count
    
    # The Manhattan Distance heuristic
    def ManhattanDist(self,start,goal):
        dist = 0
        for i in range(0,self.n):
            for j in range(0,self.n):
                 k,l = self.searchTile(goal, start[i][j])
                 dist += abs(i-k) + abs(j-l)
        return dist
    
    # A function that returns the location indices of an element
    # in a given matrix
    def searchTile(self,mat,t):
        for i in range(0,self.n):
            for j in range(0,self.n):
                if (mat[i][j] == t):
                    return i,j

    # The Nilsson Sequence heuristic
    def Nilsson(self,mat,goal):
        # Returns a list of all clockwise pairs in a given state
        def ClockwiseNeighbours(mat):
            tlist = []
            for i in range(0,self.n-1):
                t = []
                t.append(mat[0][i])
                t.append(mat[0][i+1])
                tlist.append(t)
            for i in range(0,self.n-1):
                t = []
                t.append(mat[i][2])
                t.append(mat[i+1][2])
                tlist.append(t)
            for i in reversed(range(1,self.n)):
                t = []
                t.append(mat[2][i])
                t.append(mat[2][i-1])
                tlist.append(t)
            for i in reversed(range(1,self.n)):
                t = []
                t.append(mat[i][0])
                t.append(mat[i-1][0])
                tlist.append(t)
            return tlist
        
        goal_list = ClockwiseNeighbours(goal)
        start_list = ClockwiseNeighbours(mat)
        count = 0
        # Counting the number of pairs in the current list 
        # that are not in the goal list
        for i in start_list:
          if i not in goal_list:
            count += 1
        return count + self.ManhattanDist(mat,goal)
    
    # The Out of Row and Out of Column heuristic obtained from ABSOLVER
    def OutOfRowColumn(self, mat, goal):
        count = 0
        for i in range(self.n):
            for j in range(self.n):
                k,l = self.searchTile(goal, mat[i][j])
                if(i!=k):
                    count += 1
                if(j!=l):
                    count +=1
        return count
    
    # The Linear Conflicts heuristic
    def LinearConflicts(self, mat, goal):
        # A function that returns the number of tiles that must be removed to
        # resolve linear conflicts
        def TilesToRemoveConflicts(mat_row, goal_row, ans=0):
            counts = [0 for x in range(self.n)]
            for i, t1 in enumerate(mat_row):
                if t1 in goal_row and t1 != '_':
                    goal_i = goal_row.index(t1)
                    for j, t2 in enumerate(mat_row):
                        if t2 in goal_row and t2 != '_' and i != j:
                            goal_j = goal_row.index(t2)
                            if goal_i > goal_j and i < j:
                                counts[i] += 1
                            if goal_i < goal_j and i > j:
                                counts[i] += 1
            if max(counts) == 0:
                return ans
            else:
                i = counts.index(max(counts))
                mat_row[i] = -1
                ans += 1
                return TilesToRemoveConflicts(mat_row, goal_row, ans)

        cost = self.ManhattanDist(mat, goal)
        # Preparing all rows and columns in the current state
        mat_rows = []
        mat_columns = [[] for x in range(self.n)]
        goal_rows = []
        goal_columns = [[] for x in range(self.n)]
        for i in mat:
            mat_rows.append(i)
        for i in goal:
            goal_rows.append(i)
        for j in range(self.n):
            for i in range(self.n):
                mat_columns[i].append(mat[i][j])
                goal_columns[i].append(goal[i][j])
        # Adding the minimum moves to resolve linear conflicts to the Manhattan Distance
        for i in range(self.n):
            cost += 2*TilesToRemoveConflicts(mat_rows[i], goal_rows[i])
        for i in range(self.n):
            cost += 2*TilesToRemoveConflicts(mat_columns[i], goal_columns[i])
        return cost
    
    def printMatrix(self,data):
        for i in range(0,self.n):  
            for j in range(0,self.n):  
                print(data[i][j],end=" ")
            print()
    
    def printPath(self,root):  
      if root == None:  
        return  
      self.printPath(root.parent)  
      self.printMatrix(root.data)  
      print()

    # The function that executes the puzzle
    def process(self):
        #Accepting the start and goal
        print("Enter the start state matrix \n")
        start = self.accept()
        print("Enter the goal state matrix \n")        
        goal = self.accept()

        # Using datetime to measure the time taken
        start_time = datetime.now()
        start = Node(None,start,0,0)
        start.fval = self.f(start,goal)
        # The start node goes in the open list
        self.open.append(start)
        print("\n\n")
        cur = self.open[0]
        nodes_explored = 0
        while True:
            cur = self.open[0]
            nodes_explored += 1
            #Checking if the current state and the goal state have all the elemnts on the same tiles
            #Can use any heuristic to check if all tiles are in place
            if(self.MisplacedTiles(cur.data,goal) == 0): 
                break
            for i in cur.generate_child(cur):
                i.fval = self.f(i,goal)
                self.open.append(i)
            self.closed.append(cur)
            del self.open[0]
            # Sorting the open list by the fvalue
            self.open.sort(key = lambda x:x.fval,reverse=False)
        self.printPath(cur)
        print("")
        print(f"Completed in: {datetime.now() - start_time}")
        print(f"Nodes Explored: {nodes_explored}")

In the code above, we use a standard A* search as it is optimal and guarantees to find us the shortest path if one exists. Using a greedy search, the algorithm was getting stuck in loops which is intuitive as ordering in this case was only done based on the heuristic cost which can be the same for more than one value. Hence A* is the best choice here. 

We define 5 heuristic functions:
*   Misplaced Tiles
*   Manhattan Distance
*   Nilsson's Sequence
*   Out of Row and Out of Column
*   Linear Conflicts

Choosing an appropriate test sample (which explores enough states), all the 5 heuristics were tested and the time taken to reach the goal state along with the nodes explored was noted. The results for each heuristic can be found below.



The sample used is:

Start: [ [ 8 1 3 ] , [ 4 _ 2 ] , [ 7 6 5 ] ]

Goal: [ [ 1 2 3 ] , [ 4 5 6 ] , [ 7 8 _ ] ]


More samples can be used and their results can be recorded to do a more comprehensive study. For our purpose, the above sample gives a fair idea of what we can expect.

# Misplaced Tiles

This heuristic counts the number of tiles that are not in position in any given state. This heuristic is admissible as each tile that is out of place must be moved at least once to reach the goal state.

In [ ]:
puz = Puzzle(3, "Misplaced Tiles")
puz.process()

Enter the start state matrix 

8 1 3
4 _ 2
7 6 5
Enter the goal state matrix 

1 2 3
4 5 6
7 8 _



8 1 3 
4 _ 2 
7 6 5 

8 1 3 
4 2 _ 
7 6 5 

8 1 3 
4 2 5 
7 6 _ 

8 1 3 
4 2 5 
7 _ 6 

8 1 3 
4 2 5 
_ 7 6 

8 1 3 
_ 2 5 
4 7 6 

_ 1 3 
8 2 5 
4 7 6 

1 _ 3 
8 2 5 
4 7 6 

1 2 3 
8 _ 5 
4 7 6 

1 2 3 
_ 8 5 
4 7 6 

1 2 3 
4 8 5 
_ 7 6 

1 2 3 
4 8 5 
7 _ 6 

1 2 3 
4 _ 5 
7 8 6 

1 2 3 
4 5 _ 
7 8 6 

1 2 3 
4 5 6 
7 8 _ 


Completed in: 0:01:00.810333
Nodes Explored: 19353


Time Taken :  >= 1 min

Nodes Explored :  19353

# Manhattan Distance

This is one of the most commonly used heuristics for the 8 Puzzle problem. It sums the absolute distances (in each component x and y) of a tile from its goal state.

In [ ]:
puz = Puzzle(3, "Manhattan Distance")
puz.process()

Enter the start state matrix 

8 1 3
4 _ 2
7 6 5
Enter the goal state matrix 

1 2 3
4 5 6
7 8 _



8 1 3 
4 _ 2 
7 6 5 

8 1 3 
4 2 _ 
7 6 5 

8 1 3 
4 2 5 
7 6 _ 

8 1 3 
4 2 5 
7 _ 6 

8 1 3 
4 2 5 
_ 7 6 

8 1 3 
_ 2 5 
4 7 6 

_ 1 3 
8 2 5 
4 7 6 

1 _ 3 
8 2 5 
4 7 6 

1 2 3 
8 _ 5 
4 7 6 

1 2 3 
_ 8 5 
4 7 6 

1 2 3 
4 8 5 
_ 7 6 

1 2 3 
4 8 5 
7 _ 6 

1 2 3 
4 _ 5 
7 8 6 

1 2 3 
4 5 _ 
7 8 6 

1 2 3 
4 5 6 
7 8 _ 


Completed in: 0:00:01.004994
Nodes Explored: 2355


Time Taken :  ~ 1 sec

Nodes Explored :  2355

# Nilsson's Sequence

The Nilsson's sequence evaluates the number of clockwise pairs (of tiles) that are in the current state but not in the goal state and uses this, summed with the Manhattan distance, as the heuristic. 

In [ ]:
puz = Puzzle(3, "Nilsson Sequence")
puz.process()

Enter the start state matrix 

8 1 3
4 _ 2
7 6 5
Enter the goal state matrix 

1 2 3
4 5 6
7 8 _



8 1 3 
4 _ 2 
7 6 5 

8 1 3 
4 2 _ 
7 6 5 

8 1 3 
4 2 5 
7 6 _ 

8 1 3 
4 2 5 
7 _ 6 

8 1 3 
4 2 5 
_ 7 6 

8 1 3 
_ 2 5 
4 7 6 

_ 1 3 
8 2 5 
4 7 6 

1 _ 3 
8 2 5 
4 7 6 

1 2 3 
8 _ 5 
4 7 6 

1 2 3 
_ 8 5 
4 7 6 

1 2 3 
4 8 5 
_ 7 6 

1 2 3 
4 8 5 
7 _ 6 

1 2 3 
4 _ 5 
7 8 6 

1 2 3 
4 5 _ 
7 8 6 

1 2 3 
4 5 6 
7 8 _ 


Completed in: 0:00:00.587424
Nodes Explored: 2198


Time Taken :  ~ 0.6 sec

Nodes Explored :  2198

# Out of Row and Out of Column 

Here, we simply sum the number of elements that are not in the row they should be in in the goal state and the number of elements that are not in the column they should be in in the goal state. 

In [ ]:
puz = Puzzle(3, "Out Of Row and Column")
puz.process()

Enter the start state matrix 

8 1 3
4 _ 2
7 6 5
Enter the goal state matrix 

1 2 3
4 5 6
7 8 _



8 1 3 
4 _ 2 
7 6 5 

8 1 3 
4 2 _ 
7 6 5 

8 1 3 
4 2 5 
7 6 _ 

8 1 3 
4 2 5 
7 _ 6 

8 1 3 
4 2 5 
_ 7 6 

8 1 3 
_ 2 5 
4 7 6 

_ 1 3 
8 2 5 
4 7 6 

1 _ 3 
8 2 5 
4 7 6 

1 2 3 
8 _ 5 
4 7 6 

1 2 3 
_ 8 5 
4 7 6 

1 2 3 
4 8 5 
_ 7 6 

1 2 3 
4 8 5 
7 _ 6 

1 2 3 
4 _ 5 
7 8 6 

1 2 3 
4 5 _ 
7 8 6 

1 2 3 
4 5 6 
7 8 _ 


Completed in: 0:00:00.847422
Nodes Explored: 2805


Time Taken :  ~ 0.8 sec

Nodes Explored :  2805

# Linear Conflicts

This heuristic centres around the concept of linear conflicts. 2 tiles tⱼ and tₖ are in linear conflict if tⱼ and tₖ are both in the same line, the goal positions of tⱼ and tₖ are both in that line, tⱼ is to the right of tₖ, and the goal position of tⱼ is to the left of the goal position of tₖ.

For each row/column, we count the number of tiles that must be removed from that row/column to resolve all linear conflicts. Summing up the counts for all rows and columns, we multiply by 2 to get the minimum number of additional moves needed to resolve all the linear conflicts in that state. 

This is summed with the Manhattan Distance and used as a heuristic. 

In [ ]:
puz = Puzzle(3, "Linear Conflicts")
puz.process()

Enter the start state matrix 

8 1 3
4 _ 2
7 6 5
Enter the goal state matrix 

1 2 3
4 5 6
7 8 _



8 1 3 
4 _ 2 
7 6 5 

8 1 3 
4 2 _ 
7 6 5 

8 1 3 
4 2 5 
7 6 _ 

8 1 3 
4 2 5 
7 _ 6 

8 1 3 
4 2 5 
_ 7 6 

8 1 3 
_ 2 5 
4 7 6 

_ 1 3 
8 2 5 
4 7 6 

1 _ 3 
8 2 5 
4 7 6 

1 2 3 
8 _ 5 
4 7 6 

1 2 3 
_ 8 5 
4 7 6 

1 2 3 
4 8 5 
_ 7 6 

1 2 3 
4 8 5 
7 _ 6 

1 2 3 
4 _ 5 
7 8 6 

1 2 3 
4 5 _ 
7 8 6 

1 2 3 
4 5 6 
7 8 _ 


Completed in: 0:00:00.679177
Nodes Explored: 2284


Time Taken :  ~ 0.7 sec

Nodes Explored :  2284

# Conclusion

## Time Requirements of the Heuristics
Time Taken (Misplaced Tiles) :  >= 1 min

Time Taken (Manhattan Distance) :  ~ 1 sec

Time Taken (Out of Row and Column) :  ~ 0.8 sec

Time Taken (Linear Conflicts) : ~ 0.7 sec

Time Taken (Nilsson's Sequence) :  ~ 0.6 sec

## Memory Requirements of the Heuristics

Nodes Explored (Misplaced Tiles) :  19353

Nodes Explored (Out of Row and Column) :  2805

Nodes Explored (Manhattan Distance) :  2355

Nodes Explored (Linear Conflicts) : 2284

Nodes Explored (Nilsson's Sequence) :  2198

## Discussion

As can be seen, the best possible heuristic for this case is the Nilsson's Sequence. It takes the least time to compute and explores lesser nodes to get to the solution. 

The Linear Conflicts heuristic lands a close second with the Nilsson's sequence heuristic being only slightly better. 

The Out of Row + Out of Column heuristic takes the lead over Manhattan Distance in terms of the time taken but ends up exploring much more nodes to reach the solution. In terms of nodes explored, Manhattan Distance is easily better having explored 450 nodes less than the former. 

The Misplaced Tile heuristic explored almost 9 times the number of nodes explored by Nilsson's Sequence and takes more than 1 minute to reach the goal. The other 4 heuristics discussed here clearly perform much better than Misplaced Tiles.